In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def u(x):
    return np.log(1 + x * x) + np.sin(x)

In [74]:
class Solver:

    def __init__(self, h, CFL, u):
        self.L = 1
        self.T = 1
        self.h = h
        self.CFL = CFL
        self.u = u

        self.tau = self.CFL * self.h

        self.NX = int(self.L / self.h + 1)
        self.TX = int(self.T / self.tau + 1) 

        self.x_i = np.array([(i - 1) * self.h for i in range(1, self.NX + 1)])
        self.t_i = np.array([(i - 1) * self.tau for i in range(1, self.TX + 1)])

        self.u_n_m = self.u(self.x_i)
        self.u_next_m = self.u_n_m.copy()

        self.analytic_x_i = np.linspace(0, 1, 11)
        self.analytic_u = np.sin(self.analytic_x_i) + np.log(1 + np.power((self.analytic_x_i - 1), 2))
        self.analytic_all = np.sin(self.x_i) + np.log(1 + np.power((self.x_i - 1), 2))

        self.u_result = []
        self.delta_u = []

    def solve(self):
        for t in self.t_i:
            self.u_next_m[0] = np.log(1 + t * t)
            self.u_next_m[1] = self.u_n_m[0] + self.h * (1 - (2*t)/(1+t*t)) + (np.power(self.h, 2) / 2) * ((2*(1 - t*t))/np.power((t*t + 1), 2)) + (np.power(self.h, 3) / 6) * (- 1 - (4*t*(np.power(t,2) - 3))/(np.power((1 + t*t), 3)))
            self.u_next_m[2] = self.u_n_m[0] + 2 * self.h * (1 - (2*t)/(1+t*t)) + 4 * (np.power(self.h, 2) / 2) * ((2*(1 - t*t))/np.power((t*t + 1), 2)) + 8 * (np.power(self.h, 3) / 6) * (- 1 - (4*t*(np.power(t,2) - 3))/(np.power((1 + t*t), 3)))
            for i in range(3, self.NX):
                self.u_next_m[i] = self.u_n_m[i] \
                        + (self.CFL / 6) * (2*self.u_n_m[i-3] - 9*self.u_n_m[i-2] + 18*self.u_n_m[i-1] - 11*self.u_n_m[i]) \
                        + (np.power(self.CFL, 2) / 2) * (-self.u_n_m[i-3] + 4*self.u_n_m[i-2] - 5*self.u_n_m[i-1] + 2*self.u_n_m[i]) \
                        - (np.power(self.CFL, 3) / 6) * (-self.u_n_m[i-3] + 3*self.u_n_m[i-2] - 3*self.u_n_m[i-1] + self.u_n_m[i]) \
                        + self.tau*np.cos(self.x_i[i]) + 0.5*np.power(self.tau, 2)*np.sin(self.x_i[i]) - (np.power(self.tau, 3)/6)*np.cos(self.x_i[i])
            self.u_n_m = self.u_next_m.copy()
    def display(self):
        for ind, x in enumerate(self.analytic_x_i):
            i, = np.where(np.isclose(self.x_i, x))
            self.u_result.append(self.u_n_m[i])
            self.delta_u.append(np.abs(self.u_result[ind] - self.analytic_u[ind]))
        self.df = pd.DataFrame({"x": self.analytic_x_i, "u аналит.": self.analytic_u, "u числ. ": self.u_result ,"delta u": self.delta_u})
        display(self.df)
        print(self.df['delta u'].max())
    def displayMaxErr(self):


In [76]:
s = Solver(0.01, 0.5, u)
s.solve()
s.display()

,x,u аналит.,u числ.,delta u
0,0.0,0.693147,[0.6931471805599453],[0.0]
1,0.1,0.693160,[0.6888005114204847],[0.004359750504077775]
2,0.2,0.693366,[0.6890757013672283],[0.004289871263939982]
3,0.3,0.694296,[0.6901479087089438],[0.004148417909763613]
4,0.4,0.696903,[0.6929854328777266],[0.003917609178884507]
5,0.5,0.702569,[0.6989889907816695],[0.0035800991367431756]
6,0.6,0.713062,[0.7099403622505756],[0.0031221162627331767]
7,0.7,0.730395,[0.7278575852958975],[0.0025377981828458296]
8,0.8,0.756577,[0.754742964265114],[0.0018338397876901391]
9,0.9,0.793277,[0.7922444911949996],[0.0010327492856518905]


[0.00435975]
